In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('../data/full_survey_data.csv')

/var/folders/b3/glj_22g92pl25nkdvjvxnzz00000gn/T/ipykernel_62067/4096187165.py:1: DtypeWarning: Columns (10,11,12,15,16,17,18,19,20,21,22,24,27,37,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,69,74,75,76,78,88,91,94,95,104,113,114,115,116,118,121,122,123,124,125,132,135,137,138,141,142,151,160,161,162,163,165,168,169,170,171,172,179,182,184,185,188,189,198,207,208,209,210,212,215,216,217,218,219,226,229,231,232,236,245,254,255,256,257,259,262,263,264,265,266,273,276,278,279,283,292,301,302,303,304,306,309,310,312,313,320,325,326,330,339,348,349,350,351,353,356,357,359,360,367,372,373,377,386,395,396,397,398,400,403,404,406,407,414,419,420,424,433,443,444,445,446,447,448,450,451,452,453,454,455,456,458,459,460,461,462,463,464,465,466,467,469,470,471,472,473,474,475,476,478,479,480,481,482,500,501,502,508,510,515,516,522,524,525,526,533,538,547,556,561,565,566,567,574,580,581,582,591,600,603,607,608,609,616,620,624,633,680,681,682,687,688,694,696,697,698,705,71

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54873 entries, 0 to 54872
Columns: 1599 entries, id_unique to x_duration
dtypes: float64(1027), int64(9), object(563)
memory usage: 669.4+ MB


AttributeError: 'DataFrame' object has no attribute 'RO'

# how do we ensure good quality responses are chosen (look at demographics, and metadate section)
#

columns to be used: livestock_count, livestock_died_1, year, iso_country_code, id_unique,crop_count, crop_residue_use_1, crop_name_1, livestock_heads_sheep,livestock_heads_cattle,livestock_heads_pigs,livestock_heads_goats,livestock_heads_chicken,livestock_heads_camel

In [8]:
df.id_unique.value_counts()

id_unique
bf_adn_2019_1_1      1
ng_glt_2019_133_1    1
ng_glt_2019_135_1    1
ng_glt_2019_136_1    1
ng_glt_2019_137_1    1
                    ..
gh_t10_2019_195_1    1
gh_t10_2019_196_1    1
gh_t10_2019_197_1    1
gh_t10_2019_198_1    1
zm_scn_2017_610_1    1
Name: count, Length: 54873, dtype: int64

In [9]:
df.crop_residue_use_1.value_counts()

crop_residue_use_1
feed                                        13215
soil                                        10733
soil feed                                    3077
burn                                         1619
feed compost                                 1581
                                            ...  
soil fuel burn feed compost construction        1
fuel compost feed burn                          1
other feed sell                                 1
other burn feed compost sell                    1
compost no_answer                               1
Name: count, Length: 536, dtype: int64

In [12]:
crop_1_df = df[df['crop_residue_use_1'].str.contains(r'\bburn\b', na=False)]


In [13]:
crop_1_df.crop_residue_use_1.value_counts()

crop_residue_use_1
burn                                   1619
soil burn                               787
burn feed                               463
burn fuel                               229
feed burn                               169
                                       ... 
burn feed fuel compost construction       1
fuel burn feed compost construction       1
soil feed compost burn                    1
burn feed construction compost            1
compost sell burn                         1
Name: count, Length: 195, dtype: int64

In [24]:
df.crop_count.value_counts()

crop_count
1.0     15117
2.0     15092
3.0     10535
4.0      4128
5.0      3582
0.0       531
8.0       103
6.0        84
7.0        65
10.0        1
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Index: 15117 entries, 21 to 54859
Columns: 1599 entries, id_unique to x_duration
dtypes: float64(1027), int64(9), object(563)
memory usage: 184.5+ MB


In [32]:
# test for people with only one crop
filter_df = df[df.crop_count == 1]
filter_df = filter_df.dropna(subset=['crop_residue_use_1'])

filter_df['burns_crop_1'] = filter_df['crop_residue_use_1'].str.contains(r'\bburn\b').astype(int)


In [33]:
filter_df.burns_crop_1.value_counts()

burns_crop_1
0    11125
1     1142
Name: count, dtype: int64

In [34]:
crop_1_df.crop_residue_use_1.size

4498

In [40]:

livestock_cols = [f'livestock_died_{i}' for i in range(1, 6)]

filter_df['livestock_died_total'] = filter_df[livestock_cols].sum(axis=1)


In [49]:
# List of livestock_heads_* columns
livestock_heads_cols = [
    'livestock_heads_sheep',
    'livestock_heads_cattle',
    'livestock_heads_pigs',
    'livestock_heads_goats',
    'livestock_heads_chicken',
    'livestock_heads_rabbits',
    'livestock_heads_otherpoultry',
    'livestock_heads_camel',
    'livestock_heads_guinea_pigs',
    'livestock_heads_oxen',
    'livestock_heads_buffalo',
    'livestock_heads_duck',
    'livestock_heads_geese',
    'livestock_heads_donkeys_horses',
    'livestock_heads_dogs',
    'livestock_heads_bees',
    'livestock_heads_fish'
]

# Base columns
base_cols = ['burns_crop_1', 'id_unique', 'iso_country_code', 'year', 'livestock_count', 'livestock_died_total']

# Combine and create the new DataFrame
crop_1_export_df = filter_df[base_cols + livestock_heads_cols]


In [50]:
crop_1_export_df.head()

,burns_crop_1,id_unique,iso_country_code,year,livestock_count,livestock_died_total,livestock_heads_sheep,livestock_heads_cattle,livestock_heads_pigs,livestock_heads_goats,...,livestock_heads_camel,livestock_heads_guinea_pigs,livestock_heads_oxen,livestock_heads_buffalo,livestock_heads_duck,livestock_heads_geese,livestock_heads_donkeys_horses,livestock_heads_dogs,livestock_heads_bees,livestock_heads_fish
21,0,bf_adn_2019_22_1,BF,2019,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,0,bf_adn_2019_23_1,BF,2019,1.0,3.0,1.0,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,0,bf_adn_2019_25_1,BF,2019,1.0,5.0,3.0,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,0,bf_adn_2019_26_1,BF,2019,2.0,10.0,4.0,2.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,0,bf_adn_2019_28_1,BF,2019,1.0,1.0,4.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
crop_1_export_df = crop_1_export_df.dropna(subset=['livestock_count'])

In [52]:
crop_1_export_df.head()

,burns_crop_1,id_unique,iso_country_code,year,livestock_count,livestock_died_total,livestock_heads_sheep,livestock_heads_cattle,livestock_heads_pigs,livestock_heads_goats,...,livestock_heads_camel,livestock_heads_guinea_pigs,livestock_heads_oxen,livestock_heads_buffalo,livestock_heads_duck,livestock_heads_geese,livestock_heads_donkeys_horses,livestock_heads_dogs,livestock_heads_bees,livestock_heads_fish
22,0,bf_adn_2019_23_1,BF,2019,1.0,3.0,1.0,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,0,bf_adn_2019_25_1,BF,2019,1.0,5.0,3.0,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,0,bf_adn_2019_26_1,BF,2019,2.0,10.0,4.0,2.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,0,bf_adn_2019_28_1,BF,2019,1.0,1.0,4.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,0,bf_adn_2019_33_1,BF,2019,1.0,0.0,7.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
crop_1_export_df['total_livestock'] = crop_1_export_df[livestock_heads_cols].sum(axis=1)

In [54]:
crop_1_export_df.total_livestock.value_counts()

total_livestock
0.0      1028
2.0       619
1.0       507
3.0       505
5.0       410
         ... 
320.0       1
199.0       1
305.0       1
149.0       1
319.0       1
Name: count, Length: 295, dtype: int64

In [56]:
crop_1_export_df = crop_1_export_df.drop(columns=livestock_heads_cols)
crop_1_export_df.head()

,burns_crop_1,id_unique,iso_country_code,year,livestock_count,livestock_died_total,total_livestock
22,0,bf_adn_2019_23_1,BF,2019,1.0,3.0,7.0
24,0,bf_adn_2019_25_1,BF,2019,1.0,5.0,12.0
25,0,bf_adn_2019_26_1,BF,2019,2.0,10.0,31.0
27,0,bf_adn_2019_28_1,BF,2019,1.0,1.0,17.0
32,0,bf_adn_2019_33_1,BF,2019,1.0,0.0,20.0


In [57]:
crop_1_export_df = crop_1_export_df.drop(columns='livestock_count')
crop_1_export_df.head()

,burns_crop_1,id_unique,iso_country_code,year,livestock_died_total,total_livestock
22,0,bf_adn_2019_23_1,BF,2019,3.0,7.0
24,0,bf_adn_2019_25_1,BF,2019,5.0,12.0
25,0,bf_adn_2019_26_1,BF,2019,10.0,31.0
27,0,bf_adn_2019_28_1,BF,2019,1.0,17.0
32,0,bf_adn_2019_33_1,BF,2019,0.0,20.0


In [58]:
crop_1_export_df.to_csv('../data/one_crop_owners.csv')

In [60]:
crop_1_export_df[crop_1_export_df.year == 2018].burns_crop_1.value_counts()

burns_crop_1
0    2265
1     349
Name: count, dtype: int64

['livestock_heads_sheep',
 'livestock_heads_cattle',
 'livestock_heads_pigs',
 'livestock_heads_goats',
 'livestock_heads_chicken',
 'livestock_heads_rabbits',
 'livestock_heads_otherpoultry',
 'livestock_heads_camel',
 'livestock_heads_guinea_pigs',
 'livestock_heads_oxen',
 'livestock_heads_buffalo',
 'livestock_heads_duck',
 'livestock_heads_geese',
 'livestock_heads_donkeys_horses',
 'livestock_heads_dogs',
 'livestock_heads_bees',
 'livestock_heads_fish']

In [ ]:
# create the two variables needed for the first stage for the original dataset
df['total_livestock'] = df[livestock_heads_cols].sum(axis=1)
df['livestock_died_total'] = df[livestock_cols].sum(axis=1)



In [ ]:
# correlation for the first stage
import statsmodels.formula.api as smf

# only use farmers that farm one crop 
crop_1_only_df = df[df.crop_count == 1]

first_stage = smf.ols('total_livestock ~ livestock_died_total', data=crop_1_only_df).fit()

print(first_stage.summary())


                            OLS Regression Results                            
Dep. Variable:        total_livestock   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                 1.161e-05
Date:                Thu, 12 Jun 2025   Prob (F-statistic):              0.997
Time:                        13:37:34   Log-Likelihood:            -1.3945e+05
No. Observations:               15117   AIC:                         2.789e+05
Df Residuals:                   15115   BIC:                         2.789e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               41.0097 

In [67]:
crop_1_only_df.to_csv('../data/first_stage.csv')

In [ ]:
y - herd_size, livestock_died